In [115]:
import paramiko
import os
import datetime as dt
from pandas import date_range
from tqdm import tqdm

In [116]:
DATA_PATH = r'C:\Users\Raphaël Thireau\OneDrive - versoenergy\Documents\datapipeline\process\data'
FUTURES_PATH = '/F:/EEX_Production/master/market_data/power/fr/derivatives/csv/'
SFTP_CONFIG = {
    'hostname' : 'datasource.eex-group.com',
    'port' : 22,
    'username' : 'EEX_8458',
    'password' : '4dt161SV'
}

In [117]:
def eex_connexion(sftp_config=SFTP_CONFIG):    
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect(**sftp_config)
    sftp = ssh.open_sftp()
    return ssh, sftp

In [119]:
ssh, sftp = eex_connexion()
for date in tqdm(date_range(start='2022-05-18', end='2025-02-10', inclusive='both', freq='D').date):
    if not(date.weekday() > 4):
        date_str = date.strftime('%Y%m%d')
        try:
            sftp.get(
                remotepath=os.path.join(FUTURES_PATH, str(date.year), date_str, f'PowerFutureResults_FR_{date_str}.CSV'),
                localpath=os.path.join(DATA_PATH, 'EEX', 'Power', 'Futures', f'{date_str}_Power_Futures_EEX_FR.csv'))
        except FileNotFoundError:
            with open(os.path.join(DATA_PATH, 'EEX', 'log_Power_Futures_EEX_FR.txt'), 'a') as f:
                f.write(f'{date_str}\n')
        # except OSError:
        #     ssh, sftp = eex_connexion()
        #     sftp.get(
        #         remotepath=os.path.join(FUTURES_PATH, str(date.year), date_str, f'PowerFutureResults_FR_{date_str}.CSV'),
        #         localpath=os.path.join(DATA_PATH, 'EEX', 'Power', 'Futures', f'{date_str}_Power_Futures_EEX_FR.csv'))
sftp.close()
ssh.close()

100%|██████████| 1000/1000 [12:45<00:00,  1.31it/s]
